<a href="https://colab.research.google.com/github/GrigoryBartosh/hse08_ip/blob/master/hw3_classic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd 'drive/My Drive/ip_hw_3'

/content/drive/My Drive/ip_hw_3


In [0]:
import os
import copy
import zipfile
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split

from PIL import Image

from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

PATH_DATA = 'tl-signs-hse-itmo-2020-winter.zip'

In [0]:
xs, ys = [], []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    with zip_file.open('train.csv') as file:
        train_data = pd.read_csv(file)

    for _, x in tqdm(train_data.iterrows()):
        name, lable = x['filename'], x['class_number']
        with zip_file.open(os.path.join('train', 'train', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        xs += [image]
        ys += [lable]

num_classes = max(ys) + 1

x_train, x_val, y_train, y_val = train_test_split(xs, ys, test_size=0.1)
train_data = list(zip(x_train, y_train))
val_data = list(zip(x_val, y_val))

In [0]:
# TODO

In [0]:
images = []
with zipfile.ZipFile(PATH_DATA, 'r') as zip_file:
    image_names = [f[10:] for f in zip_file.namelist() if 'test' in f]

    for name in tqdm(image_names):
        with zip_file.open(os.path.join('test', 'test', name)) as img_file:
            image = Image.open(img_file).convert('RGB')
        images += [(name, image)]

predictions = {'filename': [], 'class_number': []}
with torch.no_grad():
    for name, image in tqdm(images):
        y = 0 # TODO

        predictions['filename'] += [name]
        predictions['class_number'] += [y]

predictions = pd.DataFrame(predictions)
predictions.to_csv('test_classic.csv', index=False)